In [3]:
# default_exp engineering.nbdev

%reload_ext autoreload
%autoreload 2

# word embeding
主流的词嵌入实现方法有Mikolov的Word2Vec和斯坦福大学的Glove，也有人做过实验来比较这两种方法的优劣，并没有多大的差异。
* Word2Vec是基于预测的词向量模型，简单来说，就是给定一个词，去预测这个词周围可能出现的词，或者给定一些词来确定位于中心位置的词。
* Glove是基于统计方法的，通过对词-词共现矩阵里的非零元素进行训练。

总体来说，Word2Vec使用的是局部信息，而Glove使用的是全局信息，因此前者训练起来更慢但不占用内存，而Glove通过更多的计算资源换取训练速度上的提升。



# 评估词向量
https://blog.csdn.net/TimEcho/article/details/106281209

* 评估方式一：降维。训练好的词向量，首先要进行降维，使用工具TSNE降维到2维，这样映射到二维空间，便于观察词向量训练后是否能将同类词聚集到一起
* 评估方式二：相似度对比，选定两个词向量，计算相似度，然后与人工标注的相似度进行对比
* 评估方式三：类比。计算woman和man的距离，然后在词库中找与girl距离跟woman和man距离相近的词

## word2vec中skipgram的缺点

1. 不考虑上下文，即同一个word只有一种向量表示，没有考虑一词多义的情况  
解决方法：context-aware word embedding:ELmo或者BERT
2. 窗口长度有限，  
解决：类似于LM，使用RNN，LSTM来学习出词向量
3. 无法考虑全局，  
解决方法：全局模型（MF矩阵分解）
4. 无法有效学习低频词向量，
5. 未登录词，只在训练集出现已经训练好词向量，但是测试集没有，（OOV：out-of-vacabulary）  
4,5，使用subword embedding解决
6. 严格意义上的语序，解决：LM（RNN/LSTM）  
7. 可解释性不足,解决：非欧式空间

## subword: 
解决低频词、OOV, 根据已有词向量学习出新词的向量

这种subword的方式是这样的：加入窗口大小n=4，将每个单词分成多个部分，每个部分包含4个character，  
训练是我已然是采样skipgram模型，所以得到的词向量是subword这种形式。那么对于未登录词，完全可以通过找词库中的这种subword词向量来拼接到一个未登录词。并得到他的条件概率。  
不过这种方法不适用于中文。  
论文：enriching word embedding via wubword information

当数据量很少时，可以考虑使用这种subword

# word2vec
https://radimrehurek.com/gensim/auto_examples/tutorials/run_word2vec.html#sphx-glr-auto-examples-tutorials-run-word2vec-py

## bag-of-words model
This model transforms each document to a fixed-length vector of integers. For example, given the sentences:

    John likes to watch movies. Mary likes movies too.

    John also likes to watch football games. Mary hates football.

The model outputs the vectors:

    [1, 2, 1, 1, 2, 1, 1, 0, 0, 0, 0]

    [1, 1, 1, 1, 0, 1, 0, 1, 2, 1, 1]

Bag-of-words models are surprisingly effective, but have several weaknesses.

首先，他们会丢失所有有关单词顺序的信息：“约翰喜欢玛丽”和“玛丽喜欢约翰”对应于相同的向量。 There is a solution: bag of n-grams models consider word phrases of length n to represent documents as fixed-length vectors to capture local word order but suffer from data sparsity and high dimensionality.

其次，该模型不会尝试学习基础单词的含义，因此，向量之间的距离并不总是反映出含义上的差异。 

Word2Vec模型解决了第二个问题。

## the Word2Vec Model
Word2Vec是一种更新的模型，它使用浅层神经网络将单词嵌入到低维向量空间中。 结果是一组词向量，其中在向量空间中靠在一起的向量根据上下文具有相似的含义，而彼此远离的词向量具有不同的含义。

The are two versions of this model and Word2Vec class implements them both:

![](img/sk01.png)
### Skip-grams (SG)
The Word2Vec Skip-gram model, for example, takes in pairs (word1, word2) generated by moving a window across text data, and trains a 1-hidden-layer neural network based on the synthetic task of given an input word, giving us a predicted probability distribution of nearby words to the input. A virtual one-hot encoding of words goes through a ‘projection layer’ to the hidden layer; these projection weights are later interpreted as the word embeddings. So if the hidden layer has 300 neurons, this network will give us 300-dimensional word embeddings.

### Continuous-bag-of-words (CBOW)
Continuous-bag-of-words Word2vec is very similar to the skip-gram model. It is also a 1-hidden-layer neural network. The synthetic training task now uses the average of multiple input context words, rather than a single word as in skip-gram, to predict the center word. Again, the projection weights that turn one-hot words into averageable vectors, of the same width as the hidden layer, are interpreted as the word embeddings.

## gensim.models.Word2Vec

In [23]:
# !pip install -U gensim
!pip freeze | grep gensim

gensim==3.7.2


### 默认参数

In [7]:
from gensim.models import Word2Vec
import gensim

In [ ]:
Word2Vec()

    Word2Vec(
        sentences=None,
        corpus_file=None,
        size=100,
        alpha=0.025,
        window=5,
        min_count=5,
        max_vocab_size=None,
        sample=0.001,
        seed=1,
        workers=3,
        min_alpha=0.0001,
        sg=0,
        hs=0,
        negative=5,
        ns_exponent=0.75,
        cbow_mean=1,
        hashfxn=<built-in function hash>,
        iter=5,
        null_word=0,
        trim_rule=None,
        sorted_vocab=1,
        batch_words=10000,
        compute_loss=False,
        callbacks=(),
        max_final_vocab=None,
    )
* iter : int, optional 默认是5，要调
    Number of iterations (epochs) over the corpus.
* alpha : float, optional
    The initial learning rate.
* min_alpha : float, optional
    Learning rate will linearly drop to `min_alpha` as training progresses.
* workers : int, optional  默认是3，所以训练时要手动设置一下
    Use these many worker threads to train the model (=faster training with multicore machines).

#### min_count
min_count用于修剪内部字典。 在十亿个单词的语料库中仅出现一次或两次的单词可能是无趣的错别字和垃圾。 此外，没有足够的数据来对这些词进行任何有意义的训练，因此最好忽略它们：

min_count = 5的默认值
#### size
size是gensim Word2Vec将单词映射到的N维空间的维数（N）。

较大的值需要更多的训练数据，但可以产生更好（更准确）的模型。 合理的值在数十到数百之间。

default value of size=100
#### worker
worker，最后一个主要参数（此处为完整列表）用于训练并行化，以加快训练速度：

default value of workers=3 

### Training Your Own Model
这个语料库足够小，可以完全放入内存，但是我们将实现一个内存友好的迭代器，逐行读取它，以演示如何处理更大的语料库。

如果我们想进行任何自定义的预处理，例如 解码非标准编码，小写字母，删除数字，提取命名实体……所有这些都可以在MyCorpus迭代器中完成，而word2vec不需要知道。 所需要的就是输入产生一个句子（另一个utf8单词列表）。

In [10]:
from gensim import utils
from gensim.test.utils import datapath
class MyCorpus(object):
    """An interator that yields sentences (lists of str)."""

    def __iter__(self):
        corpus_path = datapath('lee_background.cor')
        for line in open(corpus_path):
            # assume there's one document per line, tokens separated by whitespace
            yield utils.simple_preprocess(line)

In [11]:
utils.simple_preprocess('Hundreds of people have been forced to vacate their homes')

['hundreds',
 'of',
 'people',
 'have',
 'been',
 'forced',
 'to',
 'vacate',
 'their',
 'homes']

In [12]:
from gensim.models import Word2Vec

sentences = MyCorpus()
model = Word2Vec(sentences=sentences)

In [42]:
# RuntimeError: you must first build vocabulary before training the model
# 原因:训练文本太小
# sentences = [
#     ["cat", "say", "meow"], 
#     ["cat", "say", "meow"], 
#     ["cat", "say", "meow"], 
#     ["cat", "say", "meow"], 
#     ["cat", "say", "meow"], 
#     ["cat", "say", "meow"], 
#     ["dog", "say", "woof"]]
# model = gensim.models.Word2Vec(sentences=sentences)

### 获取词向量

In [13]:
vec_king = model.wv['king']
vec_king

array([-0.03686245, -0.01609651, -0.01444744, -0.02135829, -0.00200862,
       -0.01151453, -0.01819683,  0.02734437, -0.10053819, -0.05595972,
        0.04234225,  0.04281876,  0.02587122,  0.01270954,  0.04601529,
       -0.00835075,  0.00156381, -0.01310062,  0.00457986,  0.02177754,
        0.02796555,  0.03658348, -0.00421377,  0.02298167,  0.00075962,
       -0.01811693, -0.08794788, -0.01969494, -0.01617615, -0.01301005,
        0.03622155, -0.02132978,  0.00818219, -0.01280667, -0.06247395,
       -0.02782101,  0.01052102,  0.03244915,  0.055819  ,  0.02236952,
       -0.02228366, -0.06680909, -0.03983621, -0.02890607,  0.03852952,
       -0.05652092, -0.00395071, -0.065943  ,  0.0195525 ,  0.02234451,
        0.02618746, -0.01053842, -0.01325279, -0.03100039, -0.03426435,
        0.02069941, -0.04345337,  0.02748772, -0.02539084, -0.01945122,
       -0.01035313,  0.05203196,  0.01739551, -0.02849312, -0.04863365,
       -0.01096844, -0.06444025, -0.01036584, -0.04000191,  0.04

In [38]:
model['king']  # removed

/Users/luoyonggui/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  """Entry point for launching an IPython kernel.


array([-0.01486553,  0.01823401, -0.02772865,  0.00213301,  0.0372683 ,
        0.01642814,  0.04816883, -0.05448446,  0.01253152,  0.03404462,
        0.01756212, -0.04840738, -0.03839137, -0.01304129, -0.03804687,
        0.05305994, -0.00860955, -0.00836853,  0.00134603, -0.03344008,
       -0.03039226, -0.03281286, -0.00623441, -0.02607584,  0.00408613,
        0.04711006,  0.03532708, -0.0055987 , -0.00130932,  0.02229508,
       -0.04697058, -0.02922403,  0.0131788 , -0.01622746,  0.05389683,
       -0.02905632,  0.03431457,  0.01364054,  0.00634684, -0.01930137,
       -0.01475715, -0.04367147, -0.00581169, -0.03634231,  0.01892559,
       -0.01157352,  0.02849645,  0.00408916,  0.03510711, -0.0240911 ,
       -0.03448397, -0.00420031, -0.06505472,  0.00562562, -0.03190516,
       -0.01666111, -0.04189085,  0.07402997,  0.00904749, -0.03249473,
        0.00972437,  0.03047183, -0.00820827, -0.02223486, -0.01516857,
       -0.00403475, -0.02458509,  0.02097399, -0.00187658,  0.01

#### OOV

In [34]:
try:
    vec_cameroon = model.wv['cameroon']
except KeyError:
    print("The word 'cameroon' does not appear in this model")

The word 'cameroon' does not appear in this model


### 获取最相近的词向量topk

In [15]:
r = model.wv.most_similar('king', topn=2)
r

[('highway', 0.9974420070648193), ('reported', 0.9973129630088806)]

In [20]:
r = model.wv.most_similar('king1', topn=2)
r

KeyError: "word 'king1' not in vocabulary"

In [16]:
dict(r)

{'highway': 0.9974420070648193, 'reported': 0.9973129630088806}

In [17]:
import pandas as pd

/Users/luoyonggui/anaconda3/lib/python3.7/importlib/_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
  return f(*args, **kwds)
/Users/luoyonggui/anaconda3/lib/python3.7/importlib/_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
  return f(*args, **kwds)


In [18]:
pd.Series(r)

0     (highway, 0.9974420070648193)
1    (reported, 0.9973129630088806)
dtype: object

In [19]:
pd.Series(dict(r))

highway     0.997442
reported    0.997313
dtype: float64

In [36]:
# Print the 5 most similar words to “car” or “minivan”
print(model.wv.most_similar(positive=['car', 'man'], topn=5))

[('have', 0.9996969699859619), ('five', 0.9996781349182129), ('israeli', 0.9996719360351562), ('just', 0.9996703863143921), ('other', 0.999667763710022)]


In [37]:
model.wv.doesnt_match(['fire', 'water', 'land', 'sea', 'air', 'car'])

/Users/luoyonggui/anaconda3/lib/python3.7/site-packages/gensim/models/keyedvectors.py:858: FutureWarning: arrays to stack must be passed as a "sequence" type such as list or tuple. Support for non-sequence iterables such as generators is deprecated as of NumPy 1.16 and will raise an error in the future.
  vectors = vstack(self.word_vec(word, use_norm=True) for word in used_words).astype(REAL)


'land'

### 获取词汇表

In [29]:
for i, word in enumerate(model.wv.vocab):
    if i == 10:
        break
    print(word)

hundreds
of
people
have
been
forced
to
their
homes
in


### Storing and loading models

In [30]:
pwd

'/Users/luoyonggui/PycharmProjects/nbdevlib'

In [31]:
model.save('/Users/luoyonggui/Downloads/tt.model')

/Users/luoyonggui/anaconda3/lib/python3.7/site-packages/smart_open/smart_open_lib.py:398: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


In [32]:
new_model = Word2Vec.load('/Users/luoyonggui/Downloads/tt.model')

/Users/luoyonggui/anaconda3/lib/python3.7/site-packages/smart_open/smart_open_lib.py:398: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


In [33]:
new_model.wv['king']

array([-0.01486553,  0.01823401, -0.02772865,  0.00213301,  0.0372683 ,
        0.01642814,  0.04816883, -0.05448446,  0.01253152,  0.03404462,
        0.01756212, -0.04840738, -0.03839137, -0.01304129, -0.03804687,
        0.05305994, -0.00860955, -0.00836853,  0.00134603, -0.03344008,
       -0.03039226, -0.03281286, -0.00623441, -0.02607584,  0.00408613,
        0.04711006,  0.03532708, -0.0055987 , -0.00130932,  0.02229508,
       -0.04697058, -0.02922403,  0.0131788 , -0.01622746,  0.05389683,
       -0.02905632,  0.03431457,  0.01364054,  0.00634684, -0.01930137,
       -0.01475715, -0.04367147, -0.00581169, -0.03634231,  0.01892559,
       -0.01157352,  0.02849645,  0.00408916,  0.03510711, -0.0240911 ,
       -0.03448397, -0.00420031, -0.06505472,  0.00562562, -0.03190516,
       -0.01666111, -0.04189085,  0.07402997,  0.00904749, -0.03249473,
        0.00972437,  0.03047183, -0.00820827, -0.02223486, -0.01516857,
       -0.00403475, -0.02458509,  0.02097399, -0.00187658,  0.01

# nb_export

In [5]:
from nbdev.export import *
notebook2script()

Converted 00_core.ipynb.
Converted 00_template.ipynb.
Converted algo_dl_keras.ipynb.
Converted algo_ml_shallow_tree_catboost.ipynb.
Converted algo_rs_associated_rules.ipynb.
Converted algo_rs_matrix.ipynb.
Converted algo_seq_embeding.ipynb.
Converted algo_seq_tfidf.ipynb.
Converted engineering_nbdev.ipynb.
Converted engineering_panel.ipynb.
Converted index.ipynb.


In [7]:
!nbdev_build_docs

No notebooks were modified
converting /Users/luoyonggui/PycharmProjects/nbdevlib/index.ipynb to README.md
